In [2]:
from torch import nn
from ultralytics import YOLO

import os
import json 

class Constants_Models_Weights:
    YOLOV3 = "yolov3n.pt"
    YOLOV5 = "yolov5nu.pt"
    YOLOV6 = "yolov6-n.pt"
    YOLOV8 = "yolov8n.pt"
    YOLOV9 = "yolov9c.pt"

class Constants_Models:
    YOLOV3 = "yolov3"
    YOLOV5 = "yolov5"
    YOLOV6 = "yolov6"
    YOLOV8 = "yolov8"
    YOLOV9 = "yolov9"

class Constants_Training:
    PRETRAINED = "pretrained"
    FROM_SCRATCH = "scratch"
    FINE_TUNING = "fine-tuning"

def save_json(data, filename):
    if os.path.isfile(filename):
        os.remove(filename)
    json.dump(data, open(filename, 'w'), indent=4)

def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]  # layers to freeze 
    for k, v in model.named_parameters(): 
        v.requires_grad = True  # train all layers 
        if any(x in k for x in freeze): 
            print(f'freezing {k}') 
            v.requires_grad = False 
    print(f"{num_freeze} layers are freezed.")

from torch import nn
from ultralytics import YOLO

class Yolov8(nn.Module):
    
    def __init__(self, args):
        super(Yolov8, self).__init__()

        # For RGB Datasets
        if args["training_mode"] == "pretrained":
            self.model = YOLO("yolov8n.pt")
            self.model.add_callback("on_train_start", freeze_layer)
        elif args["training_mode"] == "scratch":
            self.model = YOLO("./ultralytics/cfg/models/v8/yolov8.yaml")
        elif args["training_mode"]== "fine-tuning":
            self.model = YOLO("yolov8n.pt")

    def train(self, args, output_folder):
        
        train_results = self.model.train(data=args["data_root"], epochs=args["epochs"], imgsz=args["imgsz"], patience=args["patience"], batch=args["batch"], 
              save=args["save"], save_period=args["save_period"], device=args["device"], workers=args["workers"], project=output_folder, 
              name=args["name"], pretrained=args["pretrained"], optimizer=args["optimizer"], seed=args["seed"], cos_lr=args["cos_lr"], 
              resume=args["resume"], lr0=args["lr0"], momentum=args["momentum"], weight_decay=args["weight_decay"], val=args["val"])
        return train_results 

    def test(self, args):
        test_results = self.model.val(data=args["data_root"], device=args["device"])
        return test_results

class Yolov9(nn.Module):
    
    def __init__(self, args):
        super(Yolov9, self).__init__()

        # For RGB Datasets
        if args["training_mode"] == "pretrained":
            self.model = YOLO("yolov9c.pt")
            self.model.add_callback("on_train_start", freeze_layer)
        elif args["training_mode"] == "scratch":
            self.model = YOLO("./ultralytics/cfg/models/v9/yolov9c_multispectral.yaml")
        elif args["training_mode"]== "fine-tuning":
            self.model = YOLO("yolov9c.pt")
        
    def train(self, args, output_folder):

        train_results = self.model.train(data=args["data_root"], epochs=args["epochs"], imgsz=args["imgsz"], patience=args["patience"], batch=args["batch"], 
              save=args["save"], save_period=args["save_period"], device=args["device"], workers=args["workers"], project=output_folder, 
              name=args["name"], pretrained=args["pretrained"], optimizer=args["optimizer"], seed=args["seed"], cos_lr=args["cos_lr"], 
              resume=args["resume"], lr0=args["lr0"], momentum=args["momentum"], weight_decay=args["weight_decay"], val=args["val"],
              bands_to_apply=["RGB", "Green"]) # bands_to_apply: list of string values
        
        return train_results 


    def test(self, args):
        test_results = self.model.val(data=args["data_root"], device=args["device"], bands_to_apply=["RGB", "Green"])
        return test_results

def yolo_v(args, output_root):
    
    # Create the output directory
    if not os.path.exists(output_root):
        os.makedirs(output_root, exist_ok=True)

    # STEP 1: Save the training configuration in a file 
    save_json(args, os.path.join(output_root, "train_config.json"))
    # STEP 2: Save
    if args["model"] == Constants_Models.YOLOV9:
        model = Yolov9(args)
    elif args["model"] == Constants_Models.YOLOV8:
        model = Yolov8(args)

    # STEP 3: Train the model
    train_results = model.train(args, output_folder=output_root)
    save_json(train_results.results_dict, os.path.join(output_root, "train_results.json"))
    # STEP 4: Test the model on the test dataset
    test_results = model.test(args)
    save_json(test_results.results_dict, os.path.join(output_root, "test_results.json"))

    return test_results.results_dict


# Define the arguments manually
args = {
    'data_root': "D:/RESEARCH/pineapple_sam/multispectral_gira_10_13_mar21_lote71_5m/gira_10_13_mar21_lote71_5m_split/data.yaml",
    'output_root': './output/',
    'epochs': 20,
    'patience': 100,
    'batch': 16,
    'imgsz': 150,
    'save_period': -1,
    'device': "cpu",
    'workers': 4,
    'name': None,
    'pretrained': 1,
    'optimizer': 'auto',
    'seed': 0,
    'resume': 0,
    'lr0': 0.001,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'model': 'yolov9',
    'model_weights': 'yolov9c.pt',
    'numa': -1,
    'training_mode': "fine-tuning",  # pretrained, scratch, fine-tuning
    'model': 'yolov9',
    'save': True,
    'cos_lr': True, 
    'val': False
}
yolo_v(args, './output/')

Ultralytics YOLOv8.2.32  Python-3.12.3 torch-2.3.1 CPU (12th Gen Intel Core(TM) i5-1235U)
engine\trainer: task=detect, mode=train, model=yolov9c.pt, data=D:/RESEARCH/pineapple_sam/multispectral_gira_10_13_mar21_lote71_5m/gira_10_13_mar21_lote71_5m_split/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=150, save=True, save_period=-1, cache=False, device=cpu, workers=4, project=./output/, name=train9, exist_ok=False, pretrained=1, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=0, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=False, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_

train: Scanning D:\RESEARCH\pineapple_sam\multispectral_gira_10_13_mar21_lote71_5m\gira_10_13_mar21_lote71_5m_split\train\labels.cache... 124 images, 0 backgrounds, 0 corrupt: 100%|██████████| 124/124 [00:00<?, ?it/s]
val: Scanning D:\RESEARCH\pineapple_sam\multispectral_gira_10_13_mar21_lote71_5m\gira_10_13_mar21_lote71_5m_split\valid\labels.cache... 31 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31/31 [00:00<?, ?it/s]


Plotting labels to output\train9\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 154 weight(decay=0.0), 161 weight(decay=0.0005), 160 bias(decay=0.0)
Image sizes 160 train, 160 val
Using 0 dataloader workers
Logging results to output\train9
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/8 [00:00<?, ?it/s]

----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
--------------------

...

Exception in thread Thread-43 (plot_images):
Traceback (most recent call last):
  File "c:\Users\dnnxl\anaconda3\envs\multispectral-env\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\dnnxl\anaconda3\envs\multispectral-env\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\dnnxl\anaconda3\envs\multispectral-env\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\dnnxl\Documents\GitHub\Multispectral-yolo-ultralytics\ultralytics\utils\plotting.py", line 823, in plot_images
    mosaic[y : y + h, x : x + w, :] = images[i].transpose(1, 2, 0)
    ~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: could not broadcast input array from shape (160,192,4) into shape (160,192,3)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 1/2 [00:02<00:02,  2.83s/it]Exception in thread Thread-44 (plot_images):
Trac

----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
----------------------------------------------------------------
RGB
----------------------------------------------------------------
Green
--------------------

Exception in thread Thread-45 (plot_images):
Traceback (most recent call last):
  File "c:\Users\dnnxl\anaconda3\envs\multispectral-env\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "c:\Users\dnnxl\anaconda3\envs\multispectral-env\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\dnnxl\anaconda3\envs\multispectral-env\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\dnnxl\Documents\GitHub\Multispectral-yolo-ultralytics\ultralytics\utils\plotting.py", line 823, in plot_images
    mosaic[y : y + h, x : x + w, :] = images[i].transpose(1, 2, 0)
    ~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: could not broadcast input array from shape (160,192,4) into shape (160,192,3)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.83s/it]Exception in thread Thread-46 (plot_images):
Trac

                   all         31       1496      0.353      0.161      0.138     0.0378
Speed: 0.2ms preprocess, 88.0ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to output\train92


{'metrics/precision(B)': 0.35251110472970715,
 'metrics/recall(B)': 0.16109625668449198,
 'metrics/mAP50(B)': 0.13761840013884855,
 'metrics/mAP50-95(B)': 0.03776660870549883,
 'fitness': 0.047751787848833806}